In [1]:
import os
import json
import requests

from tqdm import tqdm

from utils import async_get, get, get_client_access_token
from utils import get_song_genres, featurize_tracks, get_dates_added

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import numpy as np

In [2]:
client_id = "d71970014054446a9f65903c9fd8afdd"
client_secret = "2be6c841192c4d4b9eda0e7faf2354c3"

access_token = get_client_access_token(client_id, client_secret)
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
access_token = "BQBnQ8SmRb-K1NyE3JmdcBsmLZDT7IO51m4g7u2-o1Ph8ktywtpz7I_q4YqYZF1HMArp9ESBAGc_GT3Hvz3dWjkKmzrfXAH3KL5DzcMb-u9UC5jqRfnDTGVD-zcRst8UpgAdliUp5X90YfDgIaaIfA3oq052gnEZGahSuN8qpU05f0LuRQ"
authorization_header = {"Authorization": "Bearer {}".format(access_token)}

In [4]:
SPOTIFY_API_BASE_URL = "https://api.spotify.com"
API_VERSION = "v1"
SPOTIFY_API_URL = "{}/{}".format(SPOTIFY_API_BASE_URL, API_VERSION)

user_profile_api_endpoint = "{}/me".format(SPOTIFY_API_URL)
user_tracks_endpoint = "{}/tracks".format(user_profile_api_endpoint)

In [5]:
# user_tracks = []
# tracks = get(requests.Session(), user_tracks_endpoint, "", authorization_header, False)
# user_tracks.append(tracks)
# i = 0
# while tracks['next']:
#     print(i, end="\r")
#     i += 1
#     tracks = get(requests.Session(), tracks['next'], "", authorization_header, False)
#     user_tracks.append(tracks)

In [6]:
queries = []
num_items = get(requests.Session(), user_tracks_endpoint, "", authorization_header, False)['total']
limit = 50
for i in range(int(num_items / limit) + 1):
    queries.append("?offset={}&limit={}".format(i * limit , limit))

In [7]:
results = await async_get(user_tracks_endpoint, queries, headers=authorization_header, verbose=True)

File                                   Completed at
https://api.spotify.com/v1/me/tracks?offset=50&limit=50                0.82s
https://api.spotify.com/v1/me/tracks?offset=200&limit=50                0.86s
https://api.spotify.com/v1/me/tracks?offset=400&limit=50                0.88s
https://api.spotify.com/v1/me/tracks?offset=450&limit=50                0.90s
https://api.spotify.com/v1/me/tracks?offset=300&limit=50                0.92s
https://api.spotify.com/v1/me/tracks?offset=150&limit=50                0.92s
https://api.spotify.com/v1/me/tracks?offset=350&limit=50                0.94s
https://api.spotify.com/v1/me/tracks?offset=100&limit=50                0.95s
https://api.spotify.com/v1/me/tracks?offset=250&limit=50                0.96s
https://api.spotify.com/v1/me/tracks?offset=500&limit=50                1.03s
https://api.spotify.com/v1/me/tracks?offset=550&limit=50                1.11s
https://api.spotify.com/v1/me/tracks?offset=600&limit=50                1.13s
https://api.s

In [8]:
user_tracks = sum([result['items'] for result in results], [])

In [9]:
track_names, track_uris, data = featurize_tracks(user_tracks, sp, verbose=True)

{'grant_type': 'client_credentials'}
https://accounts.spotify.com/api/token
{'access_token': 'BQCiqQblAjZrH3CV28aJ6cAlLQshpKNsL2zO1WgBwBAU0_cOyv0_7Q-rCyULzJA_SKfEYLHXy3saFBtHAwo', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}


In [10]:
artist_names, artist_uris, genres = get_song_genres(user_tracks, sp, verbose=True)

In [11]:
dates = get_dates_added(user_tracks)

In [12]:
user_library_data = []

num_tracks = len(user_tracks)
for i in range(num_tracks):
    track_name = track_names[i]
    date_added = dates[i]
    track_uri = track_uris[i]
    track_features = data['features'][i]
    track_genres = genres[i]
    track_artists = artist_names[i]
    user_library_data.append({ 'name' : track_name,
                               'uri' : track_uri,
                               'date' : date_added,
                               'genres' : track_genres, 
                               'artists' : track_artists,
                             })
    for feature, feature_name in zip(track_features, data['feature_names']):
        user_library_data[-1][feature_name] = feature

In [13]:
open("user_library/steven/data_user_library.json", "w").write(json.dumps(user_library_data, indent=4))

2564660